## Pipelines para pré-processamento de dados
Pipelines  são  uma  maneira  eficiente  de  encadear  várias  etapas  de  pré-processamento  de  dados  e  modelagem.  Isso  é  crucial  para  manter  o  código organizado e facilitar a reprodução de experimentos

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

# Carregar um dataset exemplo
data = load_iris()
X, y = data.data, data.target

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir o pipeline
pipeline = Pipeline([
   ('scaler', StandardScaler()),
   ('pca', PCA(n_components=3)),
   ('classifier', RandomForestClassifier())
])

# Treinar o pipeline
pipeline.fit(X_train, y_train)

# Fazer previsões
predictions = pipeline.predict(X_test)

# Mostrar previsões
print(predictions)

## Otimização de hiperparâmetros com grid search
O Grid Search é uma técnica poderosa para encontrar a combinação ideal de hiperparâmetros para um modelo, o que resulta em um desempenho otimizado.

In [1]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import load_iris

# Carregar um dataset exemplo
data = load_iris()
X, y = data.data, data.target

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir o pipeline
pipeline = Pipeline([
   ('scaler', StandardScaler()),
   ('pca', PCA(n_components=3)),
   ('classifier', RandomForestClassifier())
])

# Definir a grade de parâmetros para GridSearch
param_grid = {
   'classifier__n_estimators': [50, 100, 150],
   'classifier__max_depth': [10, 20, 30]
}

# Executar GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Mostrar os melhores parâmetros encontrados
print(grid_search.best_params_)

{'classifier__max_depth': 10, 'classifier__n_estimators': 150}


## LangChain

* Document Loaders and Utils: Essa parte do LangChain se refere aos carregadores de documentos e utilitários. Eles são responsáveis por ler, processar e carregar documentos de diversas fontes (arquivos de texto, PDFs, bancos de dados, etc.) para que possam ser utilizados nas operações de processamento de linguagem natural.

* Prompts: Prompts são instruções ou perguntas fornecidas ao modelo de linguagem para gerar respostas ou realizar uma tarefa específica. Eles são fundamentais para direcionar o comportamento do modelo de linguagem.

* Chains (ou cadeias): Elas representam uma sequência de etapas ou operações que são realizadas em conjunto para alcançar um objetivo específico. Embora o uso de um único LLM possa ser suficiente para tarefas mais simples, LangChain fornece uma interface padrão e algumas implementações comumente usadas para encadear LLMs para aplicações mais complexas, entre si ou com outros módulos especializados.

* LLM (Large Language Models): LLMs são modelos de linguagem de grande escala, como GPT-4, BERT, e outros, que são utilizados para realizar várias tarefas de processamento de linguagem natural, como tradução, sumarização, e geração de texto.

* Agents: Eles são componentes que agem de forma autônoma para realizar tarefas específicas baseadas em entradas e em modelos de linguagem. Eles podem tomar decisões e realizar ações com base em regras ou aprendizagem.


In [2]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.document_loaders import TextLoader
import os

def carregar_documentos(caminho_arquivo):
    loader = TextLoader(caminho_arquivo)
    documentos = loader.load()
    return documentos

def limpar_texto(texto):
    return texto.strip()

llm = Ollama(
    model="llama2",
    num_gpu=0,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
)

prompt_sentimento = "Analise o sentimento do seguinte texto em português: {text}"
prompt_resumo = "Gere um resumo em português para o seguinte texto: {text}"

template_sentimento = PromptTemplate(input_variables=["text"], template=prompt_sentimento)
template_resumo = PromptTemplate(input_variables=["text"], template=prompt_resumo)

chain_sentimento = LLMChain(llm=llm, prompt=template_sentimento)
chain_resumo = LLMChain(llm=llm, prompt=template_resumo)

caminho_arquivo = os.path.join(os.path.dirname(__file__), "noticias.txt")

if not os.path.exists(caminho_arquivo):
    raise FileNotFoundError(f"O arquivo {caminho_arquivo} não foi encontrado.")

documentos = carregar_documentos(caminho_arquivo)

for doc in documentos:
    texto_limpo = limpar_texto(doc.page_content)

    resultado_sentimento = chain_sentimento.run({"text": texto_limpo})
    resultado_resumo = chain_resumo.run({"text": texto_limpo})

    print(f"Notícia: {texto_limpo}")
    print(f"Sentimento: {resultado_sentimento}")
    print(f"Resumo: {resultado_resumo}")
    print("-" * 50)

/tmp/ipykernel_917839/2336885399.py:17: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(
/tmp/ipykernel_917839/2336885399.py:17: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(
/tmp/ipykernel_917839/2336885399.py:29: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_sentimento = LLMChain(llm=llm, prompt=template_sentimento)


NameError: name '__file__' is not defined